In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive
df = pd.read_csv('/content/drive/MyDrive/Laptop_price.csv')

/content/drive/MyDrive


In [ ]:
X = df.drop('Price', axis=1)
y = df['Price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
categoricalFeatures = X.select_dtypes(include=['object']).columns.tolist()
numericalFeatures = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
numericalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('mod', XGBRegressor(random_state=37))
])

In [ ]:
param = {
    'mod__n_estimators': [50, 100, 200],
    'mod__learning_rate': [0.01, 0.1, 0.2],
    'mod__max_depth': [4, 6, 8],
    'mod__gamma': [0, 0.2, 0.3],
    'mod__subsample': [0.8, 1.0]
}

In [ ]:
gridSearch = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, verbose=2)

In [ ]:
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Processor_Speed',
                                                                          'RAM_Size',
                                                                          'Storage_Capacity',
                                                                          'Screen_Size',
                                                                          'Weight']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(hand...
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=37, ...))]),
             n_jobs=-1,
             param_grid={'mod__gamma': [0, 0.2, 0.3],
                         'mod__learning_rate': [0.01, 0.1, 0.2],
                         'mod__max_depth': [4, 6, 8],
                         'mod__n_estimators': [50, 100, 200],
                         'mod__subsample': [0.8, 1.0]},
             verbose=2)

In [ ]:
mod = gridSearch.best_estimator_
bestParams = gridSearch.best_params_

In [ ]:
print("Лучшие параметры: ", bestParams)

Лучшие параметры:  {'mod__gamma': 0, 'mod__learning_rate': 0.1, 'mod__max_depth': 4, 'mod__n_estimators': 100, 'mod__subsample': 1.0}
